In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import os
import pandas as pd
from fastai.vision.all import *
import fastai ; print(fastai.__version__)

In [ ]:
path = Path('../input/cassava-leaf-disease-classification')
os.listdir(path)

In [ ]:
train_df = pd.read_csv(path/'train.csv')

In [ ]:
train_df.head()

In [ ]:
train_df['path'] = train_df['image_id'].map(lambda x:path/'train_images'/x)
train_df = train_df.drop(columns=['image_id'])
train_df = train_df.sample(frac=1).reset_index(drop=True) #shuffle dataframe
train_df.head(10)

In [ ]:
len_df = len(train_df)
print(f"There are {len_df} images")

In [ ]:
#data examples by label
train_df['label'].hist(figsize = (10, 5))

In [ ]:
item_tfms = RandomResizedCrop(460, min_scale=0.75, ratio=(1.,1.))
batch_tfms = [*aug_transforms(size=224, max_warp=0), Normalize.from_stats(*imagenet_stats)]
bs=128

In [ ]:
dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
                               valid_pct=0.2, #80-20 train-validation random split
                               seed=999, #seed
                               label_col=0, #label is in the first column of the DataFrame
                               fn_col=1, #filename/path is in the second column of the DataFrame
                               bs=bs, #pass in batch size
                               item_tfms=item_tfms, #pass in item_tfms
                               batch_tfms=batch_tfms) #pass in batch_tfms

In [ ]:
dls.show_batch()

# 1. **Model training:**

Pretrained ResNet50 with weights are added to input as a dataset, and the below code cell will allow PyTorch/fastai to find the file:

In [ ]:
# Making pretrained weights work without needing to find the default filename
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
        os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/resnet50/resnet50.pth' '/root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth'

In [ ]:
learn = cnn_learner(dls, 
                    resnet50, 
                    loss_func = LabelSmoothingCrossEntropy(), 
                    metrics = [accuracy], 
                    cbs=MixUp()).to_fp16()

In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(5,base_lr=1e-2)